# **PSI Model Synthesis Notebook for 14 Feb 2022**

Tom Schad 

----

- This notebook does an example synthesis of the global corona using a Predictive Sciences MHD model. 
- Most of the details for synthesis of any line at any time can be gleaned from this example; however, it is not currently build as its own class.  A module 'psi.py' is availabel in the examples directory to interact with the PSI models.  At some point, this may evolve into its own package. 

**A note on speed**
- This example demonstrates the forward synthesis of the Fe XIII 1074.6 nm emission line, and it uses an atomic model consisting of 50 levels. Schad & Dima (2020) discuss the relative accuracy of using models of various size, as well as the fact that computational cost increases dramatically with the number of levels included.  
- Some authors have built reduced size atomic models that attempt to emulate the behavior of a larger model atom.  While this can increase the speed of the computations, one should be careful that such models are accurate enough for your particular application.  The pyCELP ion class has been build in such a way that parallel processing is easily accomplished with the standard python tools.  
- Alternatively one could build databases in the manner of Schad & Dima (2020) to accelerate calculations over multiple datasets. 
----

In [1]:
import pycelp 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import os
os.environ["XUVTOP"] = '/usr/local/ssw/packages/chianti/dbase/'
import psi
import sunpy.coordinates.sun as sun
import sunpy.sun.constants as constants
plt.rcParams["image.origin"] = 'lower'   ## have to worry about flips? 
plt.rcParams["image.interpolation"]='nearest'
from scipy.interpolate import RegularGridInterpolator as rgi
from scipy.interpolate import interp1d
import multiprocessing
import tqdm

def euler_ry(alpha):
    '''Euler rotation matrix about y axis '''
    ry = np.array([[ np.cos(alpha), 0., np.sin(alpha)],
                   [            0., 1., 0.],
                   [-np.sin(alpha), 0., np.cos(alpha)]])
    return ry

def euler_rz(alpha):
    '''Euler rotation matrix about z axis '''
    rz = np.array([[ np.cos(alpha), -np.sin(alpha), 0.],
                   [ np.sin(alpha),  np.cos(alpha), 0.],
                   [            0.,             0., 1.]])
    return rz

## **Create an instance of the psi.Model class to load and interact with PSI coronal model data**

- Models used here were downloaded using this website from [Predictive Sciences](https://www.predsci.com/hmi/data_access.php)
- We picked the date of the US total solar eclipse (21 Aug 2017) using the med-cor-thermo2 model for this demonstration. 
- I had selected 12 UTC for the download time.  I'm not sure at this time where timestep information in the PSI models made be located.

In [2]:
## https://www.predsci.com/data/runs/cr2194-medium/hmi_mast_mas_std_0201/
## carrington rotation 2194
modelName = 'hmi__high-cor-thermo1-exp01__high-hel-poly-exp01'
corona = psi.Model('/data/tschad/psiModels/14Feb2022_thermo1_20UTC/corona/')

In [3]:
## return the instance name for some basic information on the class
corona

psi Model class
    ---------------------
    Data Directory Names: /data/tschad/psiModels/14Feb2022_thermo1_20UTC/corona/
    Number of longitude samples: 299
    Number of latitude samples: 142
    Number of radial samples: 254
    Data shape: (299, 142, 254)
    
    Variables: 
    lons -- Longitudes [rad]
    lats -- Latitudes [rad]
    rs   -- Radial samples [solar radii units]
    temp -- temperature [K]
    ne -- electron density [cm^-3]
    bx,by,bz  -- Cartesian components of magnetic field [G]
    vx,vy,vz  -- Cartesian components of velocity field [km/s]
    bmag      -- total magnetic field intensity [B]
    thetaBlocal -- location inclination of magnetic field in solar frame [rad]
    

In [185]:
kk = 25

fig,ax = plt.subplots(2,2,figsize = (8,5))
ax = ax.flatten()
im0 = ax[0].imshow(corona.bmag[:,:,kk].T*np.cos(corona.thetaBlocal[:,:,kk]).T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
ax[1].imshow(corona.thetaBlocal[:,:,kk].T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
ax[2].imshow(corona.temp[:,:,kk].T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
ax[3].imshow(corona.ne[:,:,kk].T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
labels = 'Magnetogram','Local B Inclination','Temperature','Electron Density'
for n in range(4): ax[n].set_title(labels[n])

im0.set_clim(-25,25)

cbars = []
for axi in ax:
 cax = axi.inset_axes([1.04, 0.05, 0.05, 0.95], transform=axi.transAxes)
 cbar1 = fig.colorbar(axi.get_images()[0],ax=axi,cax=cax)
 cbars.append(cbar1)
 axi.set_ylim(np.pi,0)

fig.suptitle(modelName + '\nRadial Coordinate: ' + str(corona.rs[kk]))

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This is the GONG 2194_000 synoptic map 

<img src="https://gong.nso.edu/data/magmap/QR/mqj/202202/mrmqj220221/mrmqj220221t2004c2254_000.jpg" />

In [195]:
corona.ne.shape
plt.figure()
plt.plot(corona.rs,np.max(corona.ne,axis=(0,1)))
plt.plot(corona.rs,np.min(corona.ne,axis=(0,1)))
plt.yscale('log')
plt.xscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## **Initialize the pyCELP model of the line or lines to be synthesized**

- Here we start with Fe XIII 1074 nm and keep the model fairly "small" at 50 levels.  The errors incurred by using reduced numbers of atomic levels is addressed in [Schad & Dima (2020)](https://rdcu.be/b5J2X) 


In [5]:
fe13 = pycelp.Ion('fe_13',nlevels = 50)
ln1 = fe13.get_emissionLine(10747.)
ln2 = fe13.get_emissionLine(10798.)

 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.elvlc
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.wgfa
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.scups
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.psplups
 using default abundances: /usr/local/ssw/packages/chianti/dbase/abundance/sun_photospheric_2009_asplund.abund
 reading:  /usr/local/ssw/packages/chianti/dbase/abundance/sun_photospheric_2009_asplund.abund
 testing default file: /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq
 reading:  /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq
 setting up electron collision rate factors
 setting up proton  collision rate factors
 setting up non-dipole radiative rate factors
 getting non-dipole rate factors
 setting up dipole radiative rate factors


## **Calculate the statistical equilibrium for every cell**

- Here the statistical equilibrium is calculated for the Ion for each cell in the PSI model and the results of the upper level populations, alignments, and total ion population are stored in arrays. 
- The advantage of doing it this way is that that statistical equilibrium does not change with viewing angle, and so if you want to synthesis the polarized intensities at a different viewing angle, you do not have to recalculate the statistical equilibrium. 

In [6]:
nlons,nlats,nr = corona.bx.shape

In [8]:
def work(argin): 
    
    ii,jj,kk = argin 
    
    edens = corona.ne[ii,jj,kk]
    etemp = corona.temp[ii,jj,kk]
    ht = (corona.rs[kk]-1).clip(1.e-8)
    thetab = np.rad2deg(corona.thetaBlocal[ii,jj,kk])
    
    fe13.calc_rho_sym(edens,etemp,ht, thetab)
    
    ln1 = fe13.get_emissionLine(10747.)
    ln2 = fe13.get_emissionLine(10798.)
    
    return (ii,jj,kk,ln1.C_coeff,ln1.upper_level_alignment, 
                ln2.C_coeff,ln2.upper_level_alignment)

## setup save file for repeat runs of this notebook and to not repeat the 
## statistical equilbrium calculation 

savefn = '/data/tschad/psiModels/14Feb2022_thermo1_20UTC/fe13_1074_1079_pops.npz'

if not os.path.isfile(savefn):

    ncpus = multiprocessing.cpu_count()
    arg_array = []
    for ii in range(0,nlons,1): 
        for jj in range(0,nlats,1): 
            for kk in range(0,nr,1): 
                arg_array.append([ii,jj,kk])

    ccoef_10747 = np.zeros_like(corona.bx)
    align_10747 = np.zeros_like(corona.bx)
    ccoef_10798 = np.zeros_like(corona.bx)
    align_10798 = np.zeros_like(corona.bx)

    p     = multiprocessing.Pool(processes=ncpus) 
    rs    = p.imap(work,arg_array)
    p.close()

    for res in tqdm.tqdm(rs,total = len(arg_array)): 
        ii,jj,kk,r1,a1,r2,a2 = res 
        ccoef_10747[ii,jj,kk] = r1
        align_10747[ii,jj,kk] = a1
        ccoef_10798[ii,jj,kk] = r2
        align_10798[ii,jj,kk] = a2

    np.savez(savefn,ccoef_10747=ccoef_10747,align_10747=align_10747,ccoef_10798=ccoef_10798,align_10798=align_10798)
    
else: 
    
    ## if variables not defined load 
    if "ccoef_10747" not in globals():
        print('loading: ',savefn)
        npzd = np.load(savefn)
        ccoef_10747 = npzd['ccoef_10747']
        align_10747 = npzd['align_10747']
        ccoef_10798 = npzd['ccoef_10798']
        align_10798 = npzd['align_10798']

## **Setup forward synthesis geometry**

- This is set up to synthesize a FOV from an arbitrary observer's position, def

### Observer's Position

In [9]:
## not yet sure what to use here?  How to get fractional carrington rotation of the PSI data?
crn        = sun.carrington_rotation_number('2022-02-14 20:00:00.000')
crt        = sun.carrington_rotation_time(crn)
obsLon     = sun.L0(crt).rad
obsLat     = sun.B0(crt).rad
Obs_Sun_AU = sun.earth_distance(crt).value
rsunarc    = sun.angular_radius(crt).value  ## radius of sun in arcseconds .. later replace with sun ephemeris

print('Carrington Rotation Number:       ',crn)
print('Carrington Rotation Time:       ',crt)
print('Observers Longitude [deg]:      ',np.rad2deg(obsLon))
print('Observers Latitude [deg]:       ',np.rad2deg(obsLat))
print('Observers distance to Sun [AU]: ',Obs_Sun_AU)
print('Apparent solar radius [arcsec]: ',rsunarc)

Carrington Rotation Number:        2254.2438864703195
Carrington Rotation Time:        2022-02-14 19:59:59.945
Observers Longitude [deg]:       272.2008791250737
Observers Latitude [deg]:        -6.815283900001111
Observers distance to Sun [AU]:  0.9875632519781533
Apparent solar radius [arcsec]:  971.3112266968824


In [10]:
rObs     = Obs_Sun_AU * (1.495978707e11/6.96340e8)     ## rObs is the radial position of the observer in solar radii
thetaObs = np.pi/2. - obsLat   ## Observers latitude in the PSI model geometry (0 is the NorthPole)
phiObs   = obsLon ## Observers Lon in PSI model geometry  
## cartesian position of Observer in PSI model geometery (units of solar radii)
xObs,yObs,zObs = rObs*np.sin(thetaObs)*np.cos(phiObs),rObs*np.sin(thetaObs)*np.sin(phiObs),rObs*np.cos(thetaObs)
print(xObs,yObs,zObs)

8.090140694505761 -210.50811271235773 -25.17709939268998


### Synthesized Field-of-View Size and Sampling

- The synthesis geometry will have the x axis pointed toward the observer.  Z is aligned with polar axis. 

In [37]:
yfov_range = (-2.*rsunarc,0) # (-3.*rsunarc,+3*rsunarc)
zfov_range = (-0.25*rsunarc,2.*rsunarc) # (-3.*rsunarc,+3*rsunarc)
arcsamp    = 10.  ## sampling in arcsecond

#yfov_range = (-3.*rsunarc,+3*rsunarc)
#zfov_range = (-3.*rsunarc,+3*rsunarc)
#arcsamp    = 40.  ## sampling in arcsecond

nysamp = int(np.ceil((yfov_range[1]-yfov_range[0])/arcsamp))
nzsamp = int(np.ceil((zfov_range[1]-zfov_range[0])/arcsamp))
yarc = np.linspace(yfov_range[0],yfov_range[1],nysamp)
zarc = np.linspace(zfov_range[0],zfov_range[1],nzsamp)
zza,yya  = np.meshgrid(zarc,yarc,indexing = 'ij')
rra = np.sqrt(yya**2. + zza**2.) 
mask_ondisk = 1.*(rra>rsunarc)    ## a mask where the field of view cross the solar disk

### Calculate ray samples along each line-of-sight

In [38]:
## First find the cartestian points of each line of sight in the zy plane of the observer's geometry 
xxObs = np.zeros_like(yya)
yyObs = rObs * np.tan(np.deg2rad(yya/3600.))
zzObs = rObs * np.tan(np.deg2rad(zza/3600.))

## rotate these points into the model geometry with Euler rotation
rotm = euler_ry((thetaObs-np.pi/2.)) @ euler_rz(phiObs)
## earlier I had the rotation matrix as the transpose...not sure why..incorrect?
xyz_model = rotm  @ np.stack((xxObs.flatten(),yyObs.flatten(),zzObs.flatten())) 

## NOW WITH THE XYZ_MODEL POINTS AND THE LOCATION OF THE OBSERVER,
## come up with the parametric equations for the los of sight
losvec    = np.stack((xObs-xyz_model[0,:],yObs-xyz_model[1,:],zObs - xyz_model[2,:]))
losveclen = np.linalg.norm(losvec,axis=0,ord=2,keepdims = True)
losvec    = losvec / losveclen
startpt   = xyz_model

In [39]:
## get minimum angular spacing
lons,lats,rs = corona.lons,corona.lats,corona.rs
m1 = np.min(np.abs(lons - np.roll(lons,1)))
m2 = np.min(np.abs(lats - np.roll(lats,1)))
dth = np.min([m1,m2])/2.  ## min angular spacing divided by 2 

In [40]:
nrays   = startpt.shape[1]
ray_npt = int(np.ceil(np.pi/dth))
print('Number of rays to trace: ',nrays) 
print('Max number of samples per ray: ',ray_npt)

tan_tharr = np.tan(np.linspace(-0.98*np.pi/2,0.98*np.pi/2,ray_npt))  ## limits of angles of the line-of-sight ray
rpt = np.linalg.norm(startpt,axis=0,ord=2)  ## radius of starting point in zy observer's plane 

Number of rays to trace:  42705
Max number of samples per ray:  407


In [41]:
## Magic! --- gets the cartesin coordinates in the PSI model coordinate frame for all ray samples 
## may still need to slice off the dlen portion for the trapz integration?
rays_xyz =  startpt[None,:,:] + (losvec * (tan_tharr[:,None] * rpt[None,:])[:,None,:])
rays_xyz = np.swapaxes(rays_xyz,1,2)

In [42]:
## get spherical coordinates of rays
rays_sph = np.zeros_like(rays_xyz)
rays_sph[:,:,0] = np.linalg.norm(rays_xyz,axis=2)
rays_sph[:,:,1] = np.arccos(rays_xyz[:,:,2] / rays_sph[:,:,0]) 
rays_sph[:,:,2] = np.arctan2(rays_xyz[:,:,1] , rays_xyz[:,:,0]) 
rays_sph[:,:,2][rays_sph[:,:,2] <0] += 2.*np.pi ## so that it falls in the simulation domain 

In [43]:
print(rays_sph.shape)

(407, 42705, 3)


In [44]:
rays_xyz = rays_xyz.reshape(ray_npt*nrays,3)
rays_sph = rays_sph.reshape(ray_npt*nrays,3)

## **Interpolate model variables at ray samples**

In [45]:
#### Get interpolating functions for all necessary simulation data
ccoefInt  = rgi((lons,lats,rs),ccoef_10747,method = 'linear',fill_value = 0.,bounds_error = False)
alignInt  = rgi((lons,lats,rs),align_10747,method = 'linear',fill_value = 0.,bounds_error = False)
bxInt   = rgi((lons,lats,rs),corona.bx,method = 'linear',fill_value = 0.,bounds_error = False)
byInt   = rgi((lons,lats,rs),corona.by,method = 'linear',fill_value = 0.,bounds_error = False)
bzInt   = rgi((lons,lats,rs),corona.bz,method = 'linear',fill_value = 0.,bounds_error = False)
## add velocities! 
## -- TBD 
## temperatures 
tempInt   = rgi((lons,lats,rs),corona.temp,method = 'linear',fill_value = 0.,bounds_error = False)

In [46]:
## get bx,by,bz in PSI geometry for all the ray points
## this is the step that takes the longest! 
%time bx_rays = bxInt(rays_sph[:,::-1])
%time by_rays = byInt(rays_sph[:,::-1]) 
%time bz_rays = bzInt(rays_sph[:,::-1]) 
%time ccoef_rays = ccoefInt(rays_sph[:,::-1]) 
%time align_rays = alignInt(rays_sph[:,::-1]) 
%time temp_rays = tempInt(rays_sph[:,::-1])

CPU times: user 5.06 s, sys: 1.91 s, total: 6.98 s
Wall time: 6.99 s
CPU times: user 5.07 s, sys: 1.92 s, total: 6.98 s
Wall time: 7 s
CPU times: user 5.09 s, sys: 1.91 s, total: 6.99 s
Wall time: 7.01 s
CPU times: user 5.08 s, sys: 1.94 s, total: 7.01 s
Wall time: 7.03 s
CPU times: user 5.1 s, sys: 1.92 s, total: 7.02 s
Wall time: 7.04 s
CPU times: user 5.06 s, sys: 1.88 s, total: 6.95 s
Wall time: 6.96 s


In [47]:
ccoef_rays = ccoef_rays.reshape(ray_npt,nrays)
align_rays = align_rays.reshape(ray_npt,nrays)
temp_rays = temp_rays.reshape(ray_npt,nrays)

In [48]:
rays_xyz = rays_xyz.reshape(ray_npt,nrays,3)
rays_sph = rays_sph.reshape(ray_npt,nrays,3)

In [49]:
## calculate the magnetic field inclination angle wrt the line-of-sight 
bxyzs = np.stack((bx_rays,by_rays,bz_rays)).T
blens = np.linalg.norm(bxyzs,axis =1)
bxyzs = bxyzs.reshape(ray_npt,nrays,3)
blens = blens.reshape(ray_npt,nrays)
thetaBlos = np.arccos( (np.sum(bxyzs*losvec.T[None,:,:],axis=2)/blens).clip(min = -1,max=1))
thetaBlos[np.isnan(thetaBlos)] = 0.

/tmp/ipykernel_893156/3178097367.py:6: RuntimeWarning: invalid value encountered in true_divide
  thetaBlos = np.arccos( (np.sum(bxyzs*losvec.T[None,:,:],axis=2)/blens).clip(min = -1,max=1))


In [50]:
## angle between losvec and the vector from disk center
thetaDClos = np.arccos( (np.sum(rays_xyz*losvec.T[None,:,:],axis=2)/rays_sph[:,:,0]).clip(min = -1,max=1))

In [51]:
## get projection of B onto plane perpendicular to LOS
## and the projection of DC vector onto same plane
Bperp  = bxyzs - (blens*np.cos(thetaBlos))[:,:,None] * losvec.T[None,:,:]
DCperp = rays_xyz - (rays_sph[:,:,0]*np.cos(thetaDClos))[:,:,None] * losvec.T[None,:,:]

In [52]:
del thetaDClos

In [53]:
## find angle between Bperp and DCperp, which is the azimuthal angle relative to disk center
costhetaAzi = np.sum((Bperp*DCperp),axis=2)/(np.linalg.norm(Bperp,axis = 2)*np.linalg.norm(DCperp,axis = 2))
del Bperp
del DCperp
thetaAzi = np.arccos(costhetaAzi.clip(min =-1,max=1))
thetaAzi[np.isnan(thetaAzi)] = 0.
del costhetaAzi

/tmp/ipykernel_893156/3888707345.py:2: RuntimeWarning: invalid value encountered in true_divide
  costhetaAzi = np.sum((Bperp*DCperp),axis=2)/(np.linalg.norm(Bperp,axis = 2)*np.linalg.norm(DCperp,axis = 2))


In [54]:
fig,ax = plt.subplots(2,3,figsize = (10,7),sharex=True,sharey=True)
ax = ax.flatten()
im0 = ax[0].imshow(bx_rays.reshape(ray_npt,nrays)[ray_npt//2,:].reshape(*yya.shape))
im1 = ax[1].imshow(by_rays.reshape(ray_npt,nrays)[ray_npt//2,:].reshape(*yya.shape))
im2 = ax[2].imshow(bz_rays.reshape(ray_npt,nrays)[ray_npt//2,:].reshape(*yya.shape))

im3 = ax[3].imshow(blens[ray_npt//2,:].reshape(*yya.shape))
im4 = ax[4].imshow(thetaBlos[ray_npt//2,:].reshape(*yya.shape))
im5 = ax[5].imshow(thetaAzi[ray_npt//2,:].reshape(*yya.shape))
for im in im0,im1,im2,: im.set_clim(-4,4)

labs = 'Bx (POS)','By (POS)','Bz (POS)','B (POS)','ThetaBLOS (POS)','Bazimuth (POS)'
for n in range(6): ax[n].set_title(labs[n])

cbars = []
for axi in ax:
 cax = axi.inset_axes([1.04, 0.05, 0.05, 0.95], transform=axi.transAxes)
 cbar1 = fig.colorbar(axi.get_images()[0],ax=axi,cax=cax)
 cbars.append(cbar1)

fig.suptitle(modelName + '\nMagnetic Field in the Plane of the Sky from the Observer\nComponents are in the PSI model frame') 

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## **Forward Synthesize Emission Line** 

In [199]:
geff,D_coeff,E_coeff = ln1.geff,ln1.Dcoeff,ln1.Ecoeff

In [200]:
epsI   = ccoef_rays*(1.0+(1./(2.*np.sqrt(2.)))*(3.*np.cos(thetaBlos)**2 - 1.)*D_coeff*align_rays)
epsQnr = ccoef_rays*(3./(2.*np.sqrt(2.)))*(np.sin(thetaBlos)**2)*D_coeff*align_rays
epsQ   = np.cos(2.*thetaAzi)*epsQnr
epsU   = -np.sin(2.*thetaAzi)*epsQnr
epsV   = ccoef_rays*np.cos(thetaBlos)*(1399612.2*blens)*(geff + E_coeff*align_rays)

In [201]:
## get the sample points measured in distance from the furtherest point
dray = (tan_tharr[:,None] * rpt[None,:])
dray = dray - np.nanmin(dray,axis=0)[None,:]
dray = dray * constants.radius.value*100.  ## cm 

In [202]:
epsI_int = np.trapz(epsI[:,:],dray[:,:],axis=0).reshape(*yya.shape) * mask_ondisk
epsQ_int = np.trapz(epsQ[:,:],dray[:,:],axis=0).reshape(*yya.shape) * mask_ondisk
epsU_int = np.trapz(epsU[:,:],dray[:,:],axis=0).reshape(*yya.shape) * mask_ondisk
epsV_int = np.trapz(epsV[:,:],dray[:,:],axis=0).reshape(*yya.shape) * mask_ondisk

In [203]:
fig,ax = plt.subplots(2,3,figsize = (10,7),sharex=True,sharey=True)
ax = ax.flatten()
im0 = ax[0].imshow(np.log10(epsI_int))
im1 = ax[1].imshow(np.log10(np.sqrt(epsQ_int**2. + epsU_int**2)))
im2 = ax[2].imshow(np.sqrt(epsQ_int**2. + epsU_int**2)/epsI_int)
im3 = ax[3].imshow(epsV_int)
im3.set_clim(np.percentile(epsV_int,[1,99]))
im4 = ax[4].imshow(0.5*np.arctan2(epsU_int , epsQ_int)*mask_ondisk)

labs = 'Log I','Log LinPol','LinPolFrac','epsV integ.','Azimuth Angle'
for n in range(5): ax[n].set_title(labs[n])

ax[-1].axis('off')

cbars = []
for axi in ax[:-1]:
 cax = axi.inset_axes([1.04, 0.05, 0.05, 0.95], transform=axi.transAxes)
 cbar1 = fig.colorbar(axi.get_images()[0],ax=axi,cax=cax)
 cbars.append(cbar1)

fig.suptitle(modelName + '\nLOS-integrated polarized emission coefficients') 

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/tmp/ipykernel_893156/1825237146.py:3: RuntimeWarning: divide by zero encountered in log10
  im0 = ax[0].imshow(np.log10(epsI_int))
/tmp/ipykernel_893156/1825237146.py:4: RuntimeWarning: divide by zero encountered in log10
  im1 = ax[1].imshow(np.log10(np.sqrt(epsQ_int**2. + epsU_int**2)))
/tmp/ipykernel_893156/1825237146.py:5: RuntimeWarning: invalid value encountered in true_divide
  im2 = ax[2].imshow(np.sqrt(epsQ_int**2. + epsU_int**2)/epsI_int)


In [204]:
plt.figure()
plt.imshow(epsI_int)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [214]:
## multiply by the grid size and convert units to photons cm^-2 s^-1 arcsec^-2 
sr2arcsec = (180./np.pi)**2.*3600.**2.
hh        = 6.6260755e-27                  ## erg s
phergs    = hh*(3.e8)/(10747.0e-10)      ## ergs to photons
print(phergs)  

## from line_fwhm_temp.py in synth_proj 
###[ 5303.  7892. 10746. 10798. 14301. 39343.]
## disk center photon units at 10746. 11053673.

plt.figure()
plt.imshow(epsI_int/sr2arcsec/phergs)  

1.8496535312180143e-12


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [215]:
plt.figure()
plt.imshow(epsI_int/sr2arcsec/phergs/11053673.*1e6,extent = (yya[0,0],yya[0,-1],zza[0,0],zza[-1,0]))
#plt.clim(0,500)
rsun =  971.975455850595

ypt1 =  (1.15*rsun-117.5) * np.cos(np.deg2rad(65.))
xpt1 = -(1.15*rsun-117.5) * np.sin(np.deg2rad(65.))
ypt2 =  (1.15*rsun+117.5) * np.cos(np.deg2rad(65.))
xpt2 = -(1.15*rsun+117.5) * np.sin(np.deg2rad(65.))
plt.plot([xpt1,xpt2],[ypt1,ypt2],color = 'black',lw = 2)

ypt1 =  (1.4*rsun-117.5) * np.cos(np.deg2rad(65.))
xpt1 = -(1.4*rsun-117.5) * np.sin(np.deg2rad(65.))
ypt2 =  (1.4*rsun+117.5) * np.cos(np.deg2rad(65.))
xpt2 = -(1.4*rsun+117.5) * np.sin(np.deg2rad(65.))
plt.plot([xpt1,xpt2],[ypt1,ypt2],color = 'white',lw = 2)

plt.colorbar()
plt.title(' millionths Fe XIII 1074.7 total line integrated brightness') 
plt.text(-1500,1750,'not the Peak brightness?',color = 'white')

plt.autoscale(False)
th = np.linspace(0,2.*np.pi,100)
for rr in 1.1,1.2,1.3,1.5,:
    plt.plot(rr*rsun*np.cos(th),rr*rsun*np.sin(th),color = 'white',lw = 0.8,linestyle = 'dotted',clip_on = True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
## compare the integrated line brightness to the model?

## Generate polarized spectra

- The coefficients along each ray are stored above in the epsI,epsQ,epsU,epsV variables. 
- In order to generate spectra, we need to multiple these by the appropriate normalized profile function (Gaussian) and/or the correctly scaled derivative profile 
- Then do the LOS integrated along the ray and as a function of wavelength. 

WORK TO BE DONE

-- just do one column maybe...also need to add arcseconds to the above images

## K-corona

In [ ]:
## first lets do this simply 

In [ ]:
## using equation 12.-30 to 12.40 in LD&L + Howard/Tippin to use better limb darkening functions 

In [ ]:
#lambdaCLV
uCLV = pycelp.util.uCLV
vCLV = pycelp.util.vCLV
pycelp.util.lambdaCLV[11:16]

In [ ]:
mu = np.linspace(0,1,220)
plt.figure()
for n in range(11,16): # len(uCLV)): 
    p1, = plt.plot(np.rad2deg(np.arccos(mu)), 1-uCLV[n] -vCLV[n] + uCLV[n]*mu + vCLV[n]* mu**2)
    
plt.plot(np.rad2deg(np.arccos(mu)) ,   1. - 0.6 + 0.6*mu,color = 'black')

In [ ]:
plt.figure()
plt.plot(mu , mu * np.log(1 + 1./mu))